# Save courses to RDF

## Read data from Firestore

In [1]:
# init
from os import environ
environ['FIRESTORE_EMULATOR_HOST'] = 'localhost:8080'
environ['FIREBASE_AUTH_EMULATOR_HOST'] = 'localhost:8083'
environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../serviceAccountKey.json'

import firebase_admin
from firebase_admin import firestore, initialize_app, credentials

if not firebase_admin._apps:
    cred = credentials.ApplicationDefault()
    initialize_app(cred)

db = firestore.client()

In [2]:
# get a program
siit_ref = db.collection(u'programs').document(u'siit')
program = siit_ref.get().to_dict()
program['naziv']

'softversko inzenjerstvo i informacione tehnologije'

In [3]:
# get courses
courses_snapshot = siit_ref.collection(u'courses').stream()
courses = [course.to_dict() for course in courses_snapshot]

Helper Functions

In [4]:
from re import compile
def date_to_xsd_date(string_with_date):
    p = compile('(\d{2}).(\d{2}).(\d{4}).')
    day, month, year = p.search(string_with_date).groups()
    return f'{year}-{month}-{day}'

In [15]:
from rdflib import *
from Aiiso import Aiiso
g = Graph()

UNI = Namespace('http://www.semanticweb.org/marija/ontologies/2021/university.owl')
g.bind('uni', UNI)
g.bind('dc', DC)
g.bind('foaf', FOAF)
g.bind('aiiso', Aiiso)

## Add program to rdf

In [16]:
program_node = URIRef(program['url'])

g.add((program_node, RDF.type, Aiiso.Programme))
g.add((program_node, UNI.name, Literal(program['naziv'])))
g.add((program_node, UNI.espb, Literal(program['espb'])))
g.add((program_node, UNI.semesters, Literal(program['brojSemestara'])))
g.add((program_node, UNI.ScientificField, Literal(program['naucnoStrucneOblasti'])))
g.add((program_node, UNI.LevelOfStudy, Literal(program['nivoStudija'])))
g.add((program_node, UNI.EducationalField, Literal(program['obrazovnoPolje'])))
g.add((program_node, UNI.Title, Literal(program['zvanje'])))

<Graph identifier=Nee7a9ebae07c48d8a794e88bcd8e66d8 (<class 'rdflib.graph.Graph'>)>

## Write courses to rdf

In [17]:
def addProfessors(course, course_node):
    professors = course['izvodjaciNastave']

    professors_node = URIRef(course['url'] + "/teachers")
    g.add((course_node, UNI.hasTeachers, professors_node))

    for prof in professors:
        professor_node = BNode()
        g.add((professors_node, UNI.teachers, professor_node))

        g.add((professor_node, RDF.type, FOAF.Person))
        g.add((professor_node, FOAF.name, Literal(prof['punoIme'])))
        g.add((professor_node, FOAF.title, Literal(prof['zvanje'])))
        g.add((professor_node, UNI.typeOfTeaching,
            Literal(prof['vidNastave'])))


In [22]:
g.serialize('uni.ttl')

<Graph identifier=Nee7a9ebae07c48d8a794e88bcd8e66d8 (<class 'rdflib.graph.Graph'>)>

In [18]:
def addLiterature(course, course_node):
    literature = course['literatura']
    
    # literature_node = BNode()
    literature_node = URIRef(course['url'] + "/literature")
    g.add((course_node, UNI.hasLiterature, literature_node))

    for lit in literature:
        list_item = BNode()

        g.add((literature_node, UNI.Literature, list_item))
        g.add((list_item, DC.creator, Literal(lit['autori'])))
        g.add((list_item, DC.title, Literal(lit['naziv'])))
        g.add((list_item, DC.date, Literal(lit['godina'])))
        g.add((list_item, DC.publisher, Literal(lit['izdavac'])))
        g.add((list_item, DC.language, Literal(lit['jezik'])))

In [20]:
def addEvaluation(course, course_node):
    evaluation = course['formiranjeOcene']
    
    evaluation_node = URIRef(course['url'] + "/evaluation")
    g.add((course_node, UNI.hasEvaluation, evaluation_node))

    for e in evaluation:
        list_item = BNode()

        g.add((evaluation_node, UNI.Evaluation, list_item))
        g.add((list_item, UNI.points, Literal(e['brojPoena'])))
        g.add((list_item, UNI.required, Literal(e['obavezna'])))
        g.add((list_item, UNI.preExamination, Literal(e['predispitna'])))
        g.add((list_item, UNI.activity, Literal(e['predmetnaAktivnost'])))

In [21]:
for i, course in enumerate(courses[0:3]):
    course_node = URIRef(course['url'])

    g.add((course_node, RDF.type, Aiiso.Course))
    g.add((course_node, DC.title, Literal(course['naziv'])))
    g.add((course_node, DC.description, Literal(course['sadrzaj'])))
    g.add((course_node,DC.date, Literal(date_to_xsd_date(course['osnovneInformacije']), datatype=XSD.date)))
    g.add((course_node, DC.subject, Literal(course['naucnaOblast'])))
    g.add((course_node, DC.type, Literal(course['kategorija'])))

    g.add((course_node, UNI.Purpose, Literal(course['cilj'])))
    g.add((course_node, UNI.espb, Literal(course['espb'])))
    g.add((course_node, UNI.Year, Literal(course['godina'])))
    g.add((course_node, UNI.Result, Literal(course['ishod'])))
    g.add((course_node, UNI.Methodology, Literal(course['metodologijaIzvodjenjaNastave'])))
    g.add((course_node, UNI.Semester, Literal(course['semestar'])))
    
    addProfessors(course, course_node)
    addLiterature(course, course_node)
    addEvaluation(course, course_node)


## Write tests to rdf

In [234]:
def addAnswers(question, question_node, questionuri):
    answers = question['possibleAnswers']
    
    answers_node = URIRef(questionuri + "/answers")
    g.add((question_node, uni['hasAnswers'], answers_node))
    
    for a in answers:

        g.add((answers_node, uni.Answer, Literal(a)))

In [235]:
def addCorrectAnswers(correctAnswers, question_node, questionuri):
    
    answers_node = URIRef(questionuri + "/correctAnswers")
    g.add((question_node, uni['correctAnswer'], answers_node))
    
    for a in correctAnswers:

        g.add((answers_node, uni.Answer, Literal(a)))

In [236]:
def addQuestionsAndAnswers(test, test_node, testuri):
    questions_snapshot = db.collection(u'tests').document(test['id']).collection(u'questions').stream()
    answers_snapshot = db.collection(u'tests').document(test['id']).collection(u'answers').stream()

    questions = [question.to_dict() for question in questions_snapshot]
    correctAnswers = [answer.to_dict() for answer in answers_snapshot]

    questions_node = URIRef(testuri + "/questions")
    g.add((test_node, uni['hasQuestions'], questions_node))
    
    for i, q in enumerate(questions):
        
        questionuri = testuri + "/questions/" + q['id']
        question_node = BNode()

        g.add((questions_node, uni.Question, question_node))
        g.add((question_node, uni.text, Literal(q['text'])))
        g.add((question_node, uni.points, Literal(q['maxPoints'])))
        g.add((question_node, uni.DomainProblem, Literal(q['domainProblemName'])))   
        
        addAnswers(q, question_node, questionuri)
        addCorrectAnswers(correctAnswers[i]['correctAnswers'], question_node, questionuri)


In [237]:
#get tests
tests_snapshot = db.collection(u'tests').stream()
# tests = [test.to_dict() for test in tests_snapshot]

for test_snap in tests_snapshot:
    test = test_snap.to_dict()
    test['id'] = test_snap.id
    testuri = "http://www.sotis/tests/" + test['id']
    test_node = URIRef(testuri)
    
    g.add((test_node, RDF.type, uni.Test))
    g.add((test_node, uni.name, Literal(test['name'])))
    g.add((test_node, uni.points, Literal(test['maxPoints'])))
    
    addQuestionsAndAnswers(test, test_node, testuri)

## Write takes to rdf

In [238]:
def addMyAnswers(take, take_node, takeuri):
    answers_snapshot = db.collection(u'users').document(user['uid']).collection(u'takes').document(take['id']).collection(u'myAnswers').stream()
    takeAnswers = [answer.to_dict() for answer in answers_snapshot]
    
    answers_node = URIRef(takeuri + "/my-answers")
    g.add((take_node, uni['hasAnswers'], answers_node))
    
    for a in takeAnswers:
        answer_node = BNode()

        g.add((answers_node, uni.MyAnswer, answer_node))
        g.add((answer_node, uni.points, Literal(a['points'])))
        g.add((answer_node, uni.correct, Literal(a['correct'])))
        
        for ans in a['myAnswers']:
            g.add((answer_node, uni.Answer, Literal(ans)))
        

In [239]:
# get users
users_snapshot = db.collection(u'users').stream()
users = [user.to_dict() for user in users_snapshot]

In [240]:
def addTakes(user, user_node):
    takes_snapshot = db.collection(u'users').document(user['uid']).collection(u'takes').stream()
#     takes = [take.to_dict() for take in takes_snapshot]

    for take_snap in takes_snapshot:
        take = take_snap.to_dict()
        take['id'] = take_snap.id
        takeuri = "http://www.sotis/takes/" + take['id']
        take_node = URIRef(takeuri)

        g.add((take_node, RDF.type, uni.Take))
        g.add((take_node, uni.startTime, Literal(take['startTime'])))
        g.add((take_node, uni.endTime, Literal(take['endTime'])))
        g.add((take_node, uni.points, Literal(take['points'])))
        g.add((take_node, uni.passed, Literal(take['passed'])))
        
        test_node = URIRef("http://www.sotis/tests/" + take['testId'])
        g.add((take_node, uni['of'], test_node))
        
        g.add((take_node, uni['doneBy'], user_node))
        
        addMyAnswers(take, take_node, takeuri)

In [241]:
for user in users:
    user_node = URIRef("http://www.sotis/users/" + user['uid'])
    g.add((user_node, FOAF.name, Literal(user['displayName'])))

    if(user['role'] == "student"):
        addTakes(user, user_node)

In [242]:
# g.serialize(format='turtle', destination="program-courses-tests.ttl")
g.serialize(format='turtle', destination="uni.ttl")
print(g.serialize(format='turtle'))

@prefix : <http://www.semanticweb.org/marija/ontologies/2021/university#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix schema: <http://purl.org/vocab/aiiso/schema#> .
@prefix uni: <http://www.semanticweb.org/marija/ontologies/2021/university.owl> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

schema:name rdfs:domain schema:Programme,
        :Domain,
        :Test .

<http://www.ftn.uns.ac.rs/2028112889/medjuracunarske-komunikacije-i-racunarske-mreze-1> a schema:Course ;
    dc:date "2005-10-01"^^xsd:date ;
    dc:description "Uvod. Projektovanje protokola (pojam protokola, jezici za formanu specifikaciju protokola – SDL, MSC, TTCN, UML). Metodologija realizacije protokola (jezgro, projektantski šablon, biblioteka klasa za realizaciju protokola). Protokoli za upravljanje u Internetu. Uvod u zaštitu računarskih mreža. P